In [34]:
import hashlib
from random import randint
from sage.interfaces.ecm import *

p = random_prime(1000000, lbound = 900000 )
print()
print(p)

def PK_and_SK(p):
    """
    Создание открытого ключа
    
    """
    q = ecm.factor((p-1) / 2 )[-1]
    x = randint(2, q - 1)
    g = 1
    while g == 1:
        h = randint(1, p - 2)
        g = pow(h,int((p -1) / q), p)
    y = pow(g, x, p)
    return x, q, g,  y
print(PK_and_SK(p))

def sign(p, q, x, g , mes, k):
    
    if k > q:
        return 'k > q'
    r = int(int(pow(g, k, p)) % q)
    k_minus1 = inverse_mod(k, q)
    Hm = int((hashlib.md5(mes.encode("utf-8")).hexdigest()), 16)
    s = int((k_minus1 * (Hm + x * r))) % q
    return Hm, r, s

def sign_check(r, s, q, Hm, y, x):
    w = inverse_mod(s, q)
    u1 = int((Hm * w)) % q
    u2 = int((r * w)) % q
    v  = int(pow(g, u1 + u2 * x, p)) % q
    v = int(pow(g, u1, p)*pow(y, u2, p)) % q 
    return v, r


def secret(Hm1, Hm2, s1, s2, q):
    """
    Поиск серкретного x по двум мообщениям подписанным одним (r, k)
        
    """
    
    x = (Hm1 * inverse_mod(s1, q) - Hm2 * inverse_mod(s2, q)) * inverse_mod((r1 * (inverse_mod(s2, q) - inverse_mod(s1, q))+q), q) % q
    
    return x


980687
(4254, 10007, 78177, 493346)


In [35]:
m1 = 'Успею ли я загрузить?'
m2 = 'Успел'
x, q, g,  y = PK_and_SK(p)
print(x, q, g, y)
k = 13
Hm1, r1, s1 = sign(p, q, x, g, m1, k)
Hm2, r2, s2 = sign(p, q, x, g, m2, k)
print(f'Подпись сообщения m1 = {m1}, s1 = {s1}')
print(f'Подпись сообщения m1 = {m1}, s1 = {s1}')

v1, r3 = sign_check(r1, s1, q, Hm1, y, x)
v2, r4 = sign_check(r2, s2, q, Hm2, y, x)

print(f'Проверка подписи: v1 = {v1} равно r = {r3}')
print(f'Проверка подписи: v2 = {v2} равно r = {r4}')


print(f'Боб думал, мы не узнаем секретный ключ x  = {x}, а мы его нашли :с. Он равен {secret(Hm1, Hm2, s1, s2, q)}' ) # секретный x


6293 10007 466166 924821
Подпись сообщения m1 = Успею ли я загрузить?, s1 = 9701
Подпись сообщения m1 = Успею ли я загрузить?, s1 = 9701
Проверка подписи: v1 = 1448 равно r = 1448
Проверка подписи: v2 = 1448 равно r = 1448
Боб думал, мы не узнаем секретный ключ x  = 6293, а мы его нашли :с. Он равен 6293
